In [1]:
import random
from datetime import timedelta  

# storing and anaysis
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import calmap
import plotly as py
import folium
import plotly.graph_objs as go
import plotly.graph_objects as go

In [2]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

# hide warnings
import warnings
warnings.filterwarnings('ignore')
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
from plotly import graph_objects

In [3]:
#Read DataFrame
df=pd.read_csv("covid-19_India_data.csv")
print("Shape of df: ",df.shape)

#Print Head 
df.head()

Shape of df:  (35, 8)


,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,Latitude,Longitude
0,Kerala,176,11,0,165,27-03-2020 18.07,10.8505,76.2711
1,Maharashtra,156,19,4,133,27-03-2020 18.42,19.7515,75.7139
2,Karnataka,64,5,3,56,27-03-2020 19.02,15.3173,75.7139
3,Telangana,59,1,0,58,27-03-2020 17.22,18.1124,79.0193
4,Uttar Pradesh,49,11,0,38,27-03-2020 18.42,26.8467,80.9462


# State wise created reports

In [4]:
temp_f = df.sort_values(by='Confirmed', ascending=False)
temp_f = temp_f[['State', 'Confirmed', 'Active', 'Deaths', 'Recovered']]
temp_f = temp_f.reset_index(drop=True)

temp_f.style.background_gradient(cmap="Blues", subset=['Confirmed', 'Active'])\
            .background_gradient(cmap="Greens", subset=['Recovered'])\
            .background_gradient(cmap="Reds", subset=['Deaths'])


,State,Confirmed,Active,Deaths,Recovered
0,Kerala,176,165,0,11
1,Maharashtra,156,133,4,19
2,Karnataka,64,56,3,5
3,Telangana,59,58,0,1
4,Uttar Pradesh,49,38,0,11
5,Rajasthan,45,42,0,3
6,Gujarat,44,41,3,0
7,Delhi,39,32,1,6
8,Punjab,38,37,1,0
9,Tamil Nadu,35,33,1,1


# State wise Confirmed and Deaths w.r.t dates

In [5]:
temp = df.groupby(['State', 'Last_Updated_Time'])['Confirmed', 'Deaths'].sum()
temp = temp.reset_index()

fig = px.bar(temp, x="Last_Updated_Time", y="Confirmed", color='State', orientation='v', height=600,
             title='Confirmed', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

# =========================================

fig = px.bar(temp, x="Last_Updated_Time", y="Deaths", color='State', orientation='v', height=600,
             title='Deaths', color_discrete_sequence = px.colors.cyclical.mygbm)
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()


# Donut plot of total Active,Recovered and Deaths

In [6]:
#Overall 
ac= df['Active'].sum()
rvd = df['Recovered'].sum()
dth = df['Deaths'].sum()
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.show()

# Trend of Weekly Coronavirus Cases in India

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Last_Updated_Time'], y=df['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))
fig.add_trace(go.Scatter(x=df['Last_Updated_Time'], y=df['Recovered'],
                mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=df['Last_Updated_Time'], y=df['Deaths'], 
                mode='lines+markers',marker_color='red',name='Deaths'))
fig.update_layout(title_text='Trend of Weekly Coronavirus Cases in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

# Visualization on Map

In [8]:
#Coordinates for the centre of India
lat_ind=20.5937
lon_ind=78.9629

ind_map = folium.Map(location=[lat_ind, lon_ind], tiles='cartodbpositron',
               min_zoom=4, max_zoom=6, zoom_start=4)

for i in range(df.shape[0]):
    folium.CircleMarker([df.Latitude[i], df.Longitude[i]], 
                        radius=5+(15*df.Confirmed[i]/df.Active.max()),
                        color='None',
                        fill_color='red',fill_opacity=0.4,
                        tooltip=f"Region : {df.State[i]}<br>Confirmed Cases : {df.Confirmed[i]}"
                       ).add_to(ind_map)

ind_map

In [9]:
import pandas as pd
import numpy as np 
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.graph_objects as go

# Tree Map shows State having Confirmed cases

In [10]:
fig = px.treemap(df, path=['State'], values='Confirmed', height=600, width=1000)

fig.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

# Pie plot showing Active cases w.r.t States

In [16]:
fig = px.pie(df, values = 'Active',names='State', height=600)
fig.update_traces(textposition='inside', textinfo='percent+label')

fig.update_layout(
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))

fig.show()

# Funnel area chart shows State Vs Recovered cases

In [17]:
fig = px.funnel_area(df,names='State', values='Recovered' ,height=600 )
fig.show()

# Bar plot of State showing Confirmed, Recovered and Deaths cases

In [13]:
fig = px.bar(df,x='State', y='Confirmed',
             hover_data=['Recovered', 'Deaths'], color='Recovered',
              height=400)
fig.show()

# Sunburst plot

In [14]:
fig = px.sunburst(df, path=['State', 'Deaths', 'Active'], values='Confirmed', color ='State')
fig.show()

# Rose graph / Bar Polar Graph shows Active, Deaths, Recovered cases w.r.t States

In [15]:
fig = px.bar_polar(df, r="Deaths", theta="State", hover_data = ['Recovered' , 'Active'],
                   color="State", template="ggplot2",
                   color_discrete_sequence= px.colors.sequential.Plasma_r,title = 'COVID-19 IN INDIA')
fig.show()

In [19]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Spread Chart Showing Deaths and Confirmed cases

In [20]:
df[['Deaths','Confirmed']].iplot(kind='spread')